In [1]:
import sys
import os
import cobra
import cplex 
import libsbml
import pandas as pd
import copy
from pathlib import Path
import matplotlib.pyplot as plt
import csv
import numpy as np
import seaborn as sns
from cobra import Reaction


#Change working dir first
cwd = os.getcwd()
# Split the path into a list of directories
directories = cwd.split(os.sep)
# Remove the last two directories from the list
directories = directories[:-2]
# Join the directories back into a path
new_cwd = os.sep.join(directories)
# Change the current working directory to the new path
os.chdir(new_cwd)

sys.path.append("./src")

import model_initialize as model_init
import model_manipulation  as mm


In [2]:
#Read 2-cell model
wt_model = cobra.io.read_sbml_model("./model/ios2164_2cell.xml")
trans_model = cobra.io.read_sbml_model("./model/ios2164_2cell.xml")

wt_model.solver = 'gurobi'
trans_model.solver = 'gurobi'


inf = 1e6

Set parameter Username
Academic license - for non-commercial use only - expires 2024-05-09


In [3]:
#This codeblock is to define some of the functions used for modelling

#Define linear relationship between PPFD and Cellular maintainance costs
#This formula comes from Topfer et al (2020) where she defined NGAM in a linear relationship with incident light
def compute_ngam_atp(ppfd):
    v_atp = 0.0049*ppfd + 2.7851
    return v_atp


#This function is used to set the inputs to the model used. 
def define_model_medium(model, co2, o2, ppfd, 
                        medium_dir='./misc/photo_medium.csv', no3=inf, h2o=inf, h=inf, 
                        nh4=inf, pi=inf):
    model_photo_media = mm.read_medium_csv(medium_dir, model)
    model_photo_media['EX_no3(e)'] = no3
    model_photo_media['EX_h2o(e)'] = h2o
    model_photo_media['EX_h(e)'] = h
    model_photo_media['EX_nh4(e)'] = nh4
    model_photo_media['EX_co2(e)'] = co2
    model_photo_media['EX_o2(e)'] = o2
    model_photo_media['EX_photonVis(e)'] = ppfd
    model_photo_media['EX_pi(e)'] = pi
    #Set set model medium as model
#     print('Added model medium')
    return model_photo_media

    
def turn_off_cofac_cycles(model, inact_dir='./misc/leaf_inactivated.tsv'):
    file = csv.reader(open(inact_dir), delimiter='\t')
    leaf_inactive_rxns = list()
    for rows in file:
        row_m = str()
        row_bs = str()
        for rxns in rows:
            row_m += str(rxns) + "_M"
            row_bs += str(rxns) + "_BS"
        leaf_inactive_rxns.append(row_m)
        leaf_inactive_rxns.append(row_bs)
        
    for rxns in model.reactions:
        if rxns.id in leaf_inactive_rxns:
            rxns.bounds = (0,0)
#     print('Successfully turned off cofactor-cycling reactions')

    
# #Add constraints to model
#This code block contains constraints that would simulate the assimilation rates of bs and m cells in a two-cell system (such as those seen near the midvein region of rice leaves)
# #BS photon flux must be the same/less than M flux (Adapted from B&B, 2019)
# photon_import = model.reactions.get_by_id("EX_photonVis(e)")
def add_tissue_constraints(model):
    #For input fluxes for light, we will set the flux ratio to 10:1 to reflect the anatomical proportions of our model ()
    
    BS_photon_import = model.reactions.PRISM_white_LED_BS
    M_photon_import = model.reactions.PRISM_white_LED_M

    #Set photon flux ratio to 10:1
    photon_flux = mm.set_fix_flux_ratio({M_photon_import.id:10, BS_photon_import.id:1},model)
    model.add_cons_vars(photon_flux)

    
    #UPDATE: Change CO2 intake to the M Cell instead rather than set a ratio, which is a better assumption overall. Assume na lang that external gasses are assimilated
    #Via the M cell.
    #From Morrison et al 2005 -- Lateral diffusion of Gases is unlikely to support photosynthesis due to the
    #assimilation of diffused CO2 in tissues prior to BS//
    model.reactions.CO2tex_BS.bounds = (0,0)
    model.reactions.O2tex_BS.bounds = (0,0)
    
    #UPDATE: This assumption does not hold considering that recent transcriptomic analysis confirms that 
    #the bundle sheath is involved in the assimilation of inorganic nutrients, including nitrogen (nitrates/ammonia), and 
    #Sulfates. In turn, this will be implemented by simply setting the exchanges to the M cell to 0. (Hua et al, 2021)
    model.reactions.SO3tex_M.bounds = (0,0)
    model.reactions.SO4tex_M.bounds = (0,0)
    model.reactions.NH4tex_M.bounds = (0,0)
    model.reactions.NO3tex_M.bounds = (0,0)
    
    #Model will also constraint H2O input to BS cell only as it is also assumed that BS tissue in rice is specialized for H2O transport (Hua et al. 2021)
    #The model also will allow one-way flux of H2O out of the M cell in the form of evaporation (shown by -inf lower bound for the reaction)
    model.reactions.H2Otex_M.bounds = (-inf, 0)
    
    #need to turn off HCO import as the model incorrectly transfers fixed HCO to the BS cell via the common pool compartment
    model.reactions.HCO3tex_M.bounds = (0,0)
    model.reactions.HCO3tex_BS.bounds = (0,0)
    
    #No constraints will be implemented for H+ availability allowing the model to use protons on-demand.

    # #This code block contains constraints specific for enzyme rate constraints
    #This approach is derived from Bogart & Myers (2016) where they constrained the enzyme rate 
    #fluxes in each of the 2-cell segments to a specific upper bound while keeping the lower bound
    #At 0. For reversible reactions the lower bounds are set to the same value
    
def add_enzyme_constraints(model, 
                           wt_pepc = 0, 
                           wt_mdh = 11.18, 
                           wt_nadp_me = 0.14, 
                           wt_ppdk=0.31,
                          wt_CA=7.5):
    
    #PEPC constraint (Reaction id: PPCc)
    #Need to constrain it to 0 since reaction is only detected in Vascular tissue
    pepc_BS = model.reactions.PPCc_BS
    pepc_M = model.reactions.PPCc_M
    
    pepc_BS.bounds = (0,0)
    pepc_M.bounds = (0,0)

    #PPDK constraints (Reaction id: PPDKs) (note that this is found in the chloroplast?) 
    #Not detected via immunolocalization but enzyme activity is detected

    ppdks_BS = model.reactions.PPDKs_BS
    ppdks_M = model.reactions.PPDKs_M
    ppdkc_BS = model.reactions.PPDKc_BS
    ppdkc_M = model.reactions.PPDKc_M
    wt_ppdks_cons = model.problem.Constraint(ppdks_BS.flux_expression 
                                             + ppdks_M.flux_expression
                                             + ppdkc_BS.flux_expression
                                             + ppdkc_M.flux_expression, 
                                             lb = 0, ub = wt_ppdk)
    wt_ppdks_cons.name = 'wt_ppdks_cons'
    model.add_cons_vars(wt_ppdks_cons)
    #Malate Dehydrogenase 
    #Only mitochondrial in WT Rice M cells
    mdhm_M = model.reactions.MDHm_M


    wt_mdh_cons = model.problem.Constraint(mdhm_M.flux_expression,
                                           lb= 0, ub=wt_mdh)
    wt_mdh_cons.name = "wt_mdh_cons"
    model.add_cons_vars(wt_mdh_cons)

    #NADP-ME (Since no signal is detected in WT, no locational constraints are imposed)
    #Let's see if I can force it to have a small amount of flux 
    nadp_me_M = model.reactions.MDHys_M
    nadp_me_BS = model.reactions.MDHys_BS

    wt_nadpme_cons = model.problem.Constraint(nadp_me_M.flux_expression
                                             + nadp_me_BS.flux_expression,
                                             lb= 0, ub=wt_nadp_me)
    wt_nadpme_cons.name = "wt_nadpme_cons"
    model.add_cons_vars(wt_nadpme_cons)


    #I should add constraints for Carbonic Anhydrase
    #I should constrain it to 0.4 ubar, which would constitute ambient CO2 partial pressure
    #Flux is reversible so constraints are bi-directional


    hco3es_m = model.reactions.HCO3Es_M.flux_expression
    hco3ec_m = model.reactions.HCO3Ec_M.flux_expression
    hco3em_m = model.reactions.HCO3Em_M.flux_expression
    hco3es_bs = model.reactions.HCO3Es_BS.flux_expression
    hco3ec_bs = model.reactions.HCO3Ec_BS.flux_expression
    hco3em_bs = model.reactions.HCO3Em_BS.flux_expression

    ca_cons = model.problem.Constraint(hco3es_m + hco3ec_m + hco3em_m 
                                       + hco3es_bs + hco3ec_bs + hco3em_bs,
                                      lb = -wt_CA, ub = wt_CA)
    ca_cons.name = 'Carbonic_anhydrase_constraint'
    model.add_cons_vars(ca_cons)
    #Rbcl constaints
    #Retrieve flux expressions oof each RBCl reaction
    rbpc_M = model.reactions.RBPCs_M.flux_expression
    rbpc_BS = model.reactions.RBPCs_BS.flux_expression
    rbpo_M = model.reactions.RBPOs_M.flux_expression
    rbpo_BS = model.reactions.RBPOs_BS.flux_expression

    #Constraint such that it is limited to 132 umol m-2 s-1
    rbcl_vcmax_cons = model.problem.Constraint(rbpc_M + rbpc_BS, lb = 0, ub= 132)
    rbcl_vcmax_cons.name='rbcl_vcmax_cons'
    model.add_cons_vars(rbcl_vcmax_cons)
    #Constraints for rbcl flux such that v_c/v_o = 3 or higher.
    rbcl_vcvo = model.problem.Constraint(3*(rbpo_M + rbpo_BS) 
                                         - 1*(rbpc_M + rbpc_BS),
                                         lb=0,ub=1000)
    rbcl_vcvo.name = 'rbcl_vc/vo_ratio'
    model.add_cons_vars(rbcl_vcvo)

    #Turn off the RBPC2s reactions since we already defined the constraints above
    model.reactions.RBPC2s_M.bounds = (0,0)
    model.reactions.RBPC2s_BS.bounds = (0,0)
    


#This function takes two arguments: the model and the maximal  ppfd input to the system
def add_ngam_cons(model, ppfd): 
    ngam_atp_m = mm.get_rxn(model, 'ngam_atp_c_M')
    ngam_atp_bs = mm.get_rxn(model, 'ngam_atp_c_BS')
    ngam_atp_m.bounds = (0,inf)
    ngam_atp_bs.bounds = (0,inf)
    ngam_ratio = mm.set_fix_flux_ratio({ngam_atp_m.id:10, ngam_atp_bs.id:1}, model)
    ngam_ratio.name = 'ngam_BS/M_ratio'
    model.add_cons_vars(ngam_ratio)

    #Retrieve NGAM reactions
    ngam_nadphox_c_M = mm.get_rxn(model, 'ngam_nadphox_c_M')
    ngam_nadphox_s_M = mm.get_rxn(model, 'ngam_nadphox_s_M')
    ngam_nadphox_m_M = mm.get_rxn(model, 'ngam_nadphox_m_M')
    ngam_nadphox_c_BS = mm.get_rxn(model, 'ngam_nadphox_c_BS')
    ngam_nadphox_s_BS = mm.get_rxn(model, 'ngam_nadphox_s_BS')
    ngam_nadphox_m_BS = mm.get_rxn(model, 'ngam_nadphox_m_BS')


    #Set Fixed fluxes
    nadphox_c_s_M = mm.set_fix_flux_ratio({ngam_nadphox_c_M.id:1, ngam_nadphox_s_M.id:1},model)
    nadphox_c_s_M.name = "nadphox_cs_ratio_M"
    nadphox_s_m_M = mm.set_fix_flux_ratio({ngam_nadphox_s_M.id:1, ngam_nadphox_m_M.id:1}, model)
    nadphox_s_m_M.name = "nadphox_sm_ratio_M"

    nadphox_c_s_BS = mm.set_fix_flux_ratio({ngam_nadphox_c_BS.id:1, ngam_nadphox_s_BS.id:1},model)
    nadphox_c_s_BS.name = "nadphox_cs_ratio_BS"
    nadphox_s_m_BS = mm.set_fix_flux_ratio({ngam_nadphox_s_BS.id:1, ngam_nadphox_m_BS.id:1}, model)
    nadphox_s_m_BS.name = "nadphox_sm_ratio_BS"

    #Add constraints
    model.add_cons_vars(nadphox_c_s_M)
    model.add_cons_vars(nadphox_s_m_M)
    model.add_cons_vars(nadphox_c_s_BS)
    model.add_cons_vars(nadphox_s_m_BS)

    #Retrieve flux expressionns
    fex_nadphox_c_M =  mm.get_flux_exp(model, ngam_nadphox_c_M)
    fex_nadphox_s_M = mm.get_flux_exp(model, ngam_nadphox_s_M)
    fex_nadphox_m_M = mm.get_flux_exp(model, ngam_nadphox_m_M)

    fex_nadphox_c_BS =  mm.get_flux_exp(model, ngam_nadphox_c_BS)
    fex_nadphox_s_BS =  mm.get_flux_exp(model, ngam_nadphox_s_BS)
    fex_nadphox_m_BS =  mm.get_flux_exp(model, ngam_nadphox_m_BS)

    fex_atp_c_M = mm.get_flux_exp(model, ngam_atp_m)
    fex_atp_c_BS =  mm.get_flux_exp(model, ngam_atp_bs)

    #Set the constraint between ATP:NADPH NGAM to 3:1
    nadphox_atpase = model.problem.Constraint(3*(fex_nadphox_c_M + fex_nadphox_s_M + fex_nadphox_m_M
                                                       + fex_nadphox_c_BS + fex_nadphox_s_BS + fex_nadphox_m_BS) 
                                         - 1*(fex_atp_c_M + fex_atp_c_BS),
                                         lb=0,ub=0)
    nadphox_atpase.name = "nadphox_atpase_ratio"
    model.add_cons_vars(nadphox_atpase)
    #Compute NGAM value and add constraint as a lower bound/upper bound to model
    ngam_value = compute_ngam_atp(ppfd)
    ngam_cons = model.problem.Constraint(fex_atp_c_M + 
                                        fex_atp_c_BS, lb=ngam_value, ub=ngam_value)
    ngam_cons.name = 'NGAM_ATP_constraint'
    model.add_cons_vars(ngam_cons)
    
#This code  block gives a snapshot of the relevant fluxes on each of the cell types based on the saved sample_fluxes values above

def print_summary(sample_fluxes_df):
    print('rbcl M cell: ', sample_fluxes['RBPCs_M'], 'rbcl BS cell: ',sample_fluxes['RBPCs_BS'])
    print('rbcl M cell (photorespiration)', sample_fluxes['RBPOs_M'], 'rbcl BS cell (PR)', sample_fluxes['RBPOs_BS'])
    print('vc/vo M:', sample_fluxes['RBPCs_M']/sample_fluxes['RBPOs_M'], 'vc/vo BS:', sample_fluxes['RBPCs_BS']/sample_fluxes['RBPOs_BS'])
    print('RBPC2s_M', sample_fluxes['RBPC2s_M'], 'RBPC2s_BS', sample_fluxes['RBPC2s_BS'])
    print('PEPC M', sample_fluxes['PPCc_M'], 'PEPC BS', sample_fluxes['PPCc_BS'])
    print('Carbonic Anhydrase (Cytosolic) M', sample_fluxes['HCO3Ec_M'], 'Carbonic Anhydrase (Cytosolic) BS', sample_fluxes['HCO3Ec_BS'])
    print('NADP-ME M', sample_fluxes['MDHys_M'], 'NADP-ME BS', sample_fluxes['MDHys_BS'])
    print('Biomass M: ', sample_fluxes['Straw_Biomass_M'], 'Biomass BS', sample_fluxes['Straw_Biomass_BS'])
    print('Phloem M: ', sample_fluxes['DM_Phloem_M'], 'Phloem BS', sample_fluxes['DM_Phloem_BS'])
    print('co2 consumption M', sample_fluxes['CO2tex_M'], 'co2 consumption BS', sample_fluxes['CO2tex_BS'])
    print('o2 consumption M', sample_fluxes['O2tex_M'], 'o2 consumption BS', sample_fluxes['O2tex_BS'])
    print('Photosystem II M', sample_fluxes['PSIINC_M'], 'PSII BS', sample_fluxes['PSIINC_BS'])
    print('PSI M', sample_fluxes['PSIMR_M'], 'PSI BS', sample_fluxes['PSIMR_BS'])
    print('PPFD M: ', sample_fluxes['PRISM_white_LED_M'], 'PPFD BS: ', sample_fluxes['PRISM_white_LED_BS'])
    print('ATP synthesis (stromal) M', sample_fluxes['ATPSs_M'], 'ATP synthase (mit) M', sample_fluxes['ATPSm_M'])
    pd_rxn = [x for x in model.reactions if "pd" in x.id and "h2o" not in x.id]
    pd_abs_flux = 0
    for pds in pd_rxn:
        pd_abs_flux += abs(sample_fluxes[pds.id])
    
    print('pd_abs_flux: ', pd_abs_flux)
    
#initialize list of transgenic reactions to add  to model

def add_trans_reactions(model):
    '''
    This function is used to add a number of new tissue-specific reactions that were not present in the
    original model to facilitate modelling of the transgenic C4 rice
    '''
    trans_list = list()
    #Transgenic PEPC copy
    #PEPC = Chloroplastic in M & V (rxn id: PPCc)
    trans_ppcs = Reaction('trans_PPCs_M')
    trans_ppcs.name = "Phosphoenolpyruvate carboxylase, plastidic (Transgenic)"
    
    pep_s0 = model.metabolites.pep_s0
    hco3_s0 = model.metabolites.hco3_s0
    oaa_s0 = model.metabolites.oaa_s0
    pi_s0 = model.metabolites.pi_s0


    #Add metabolites, bounds, and subsystem
    trans_ppcs.add_metabolites({hco3_s0:-1, pep_s0:-1, oaa_s0:1, pi_s0:1})
    trans_ppcs.bounds= model.reactions.PPCc_M.bounds
    trans_ppcs.subsystem = model.reactions.PPCc_M.subsystem

    trans_list.append(trans_ppcs)


    #Transgenic PPDK Copy
    #Since it already exists I'll just copy and readd it
    trans_ppdks_m = Reaction('trans_PPDKs_M')
    trans_ppdks_m.add_metabolites(model.reactions.PPDKs_M.metabolites)
    trans_ppdks_m.bounds = model.reactions.PPDKs_M.bounds
    trans_ppdks_m.name = "Pyruvate phosphate dikinase, plastidic (Transgenic)"

    trans_ppdks_bs = Reaction('trans_PPDKs_BS')
    trans_ppdks_bs.add_metabolites(model.reactions.PPDKs_BS.metabolites)
    trans_ppdks_bs.bounds = model.reactions.PPDKs_BS.bounds
    trans_ppdks_bs.name = "Pyruvate phosphate dikinase, plastidic (Transgenic)"

    trans_list.append(trans_ppdks_m)
    trans_list.append(trans_ppdks_bs)


    #Transgenic NADP-ME
    #NADP-ME = Mitochondrial in M
    trans_nadp_me = Reaction('trans_MDHym_M')

    #retrieve reactants
    mal_m0 = model.metabolites.get_by_id('mal-L_m0')
    nadp_m0 = model.metabolites.nadp_m0
    h_m0 = model.metabolites.h_m0
    nadph_m0 = model.metabolites.nadph_m0
    co2_m0 = model.metabolites.co2_m0
    pyr_m0 = model.metabolites.pyr_m0

    #Add to rxn
    trans_nadp_me.add_metabolites({mal_m0:-1, nadp_m0:-1, co2_m0:1, nadph_m0:1, pyr_m0:1})
    #Add bounds
    trans_nadp_me.bounds=(0, inf)

    trans_list.append(trans_nadp_me)


    #Malate Dehydrogenase, mitochondrial (M cell)
    trans_MDHm_M = Reaction('trans_MDHm_M')
    trans_MDHm_M.name = 'Malate Dehydrogenase, Mitochondrial'
    trans_MDHm_M.add_metabolites(model.reactions.MDHm_M.metabolites)
    trans_MDHm_M.subsystem = model.reactions.MDHm_M.subsystem

    trans_list.append(trans_MDHm_M)

    #Malate dehydrogenase, plastidic (M cell)
    trans_MDHs_M = Reaction('trans_MDHs_M')
    trans_MDHs_M.name = 'Malate Dehydrogenase, Plastidic'
    trans_MDHs_M.add_metabolites(model.reactions.MDHs_M.metabolites)
    trans_MDHs_M.subsystem = model.reactions.MDHs_M.subsystem

    trans_list.append(trans_MDHs_M)

    #Malate dehydrogenase, plastidic(BS Cell)
    trans_MDHs_BS = Reaction('trans_MDHs_BS')
    trans_MDHs_BS.name = 'Malate Dehydrogenase, Plastidic'
    trans_MDHs_BS.add_metabolites(model.reactions.MDHs_BS.metabolites)
    trans_MDHs_BS.subsystem = model.reactions.MDHs_BS.subsystem

    trans_list.append(trans_MDHs_BS)


    #Trans CA
    #Cytosolic in M
    trans_hco3ec_M = Reaction('trans_hco3ec_M')
    trans_hco3ec_M.name = 'carbonic anhydrase, cytosolic'
    trans_hco3ec_M.add_metabolites(model.reactions.HCO3Ec_M.metabolites)
    trans_hco3ec_M.bounds = model.reactions.HCO3Ec_M.bounds

    trans_hco3ec_M.subsystem = model.reactions.HCO3Ec_M.subsystem
    trans_list.append(trans_hco3ec_M)


    #Bulk add to model
    model.add_reactions(trans_list)
    
    model.repair()
####ADDING TRANS CONSTRAINTS

def add_trans_constraints(model,
                         trans_pepc_rates = 7.01,
                         trans_ppdks_rates = 3.66,
                         trans_mdh_rates = 152.87,
                         trans_nadp_me_rates = 0.60,
                         trans_CA_rates = 8):
    '''
    This function is used to add another layer of constraints to parametize model based on the
    Enzyme reaction rates assayed from Ermakova et al (2021) where the locations are based on the 
    each of the transgenic enzyme's tissue-specific localizations. 
    '''
    
    #PEPC constraint
    wt_PPCc_M = mm.get_rxn(model, 'PPCc_M')
    wt_PPCc_BS = mm.get_rxn(model, 'PPCc_BS')
    trans_PPCs_M = mm.get_rxn(model, 'trans_PPCs_M')                           
    trans_PEPC_cons = model.problem.Constraint(trans_PPCs_M.flux_expression
                                            +wt_PPCc_BS.flux_expression 
                                            + wt_PPCc_M.flux_expression, 
                                            lb = 0, ub = trans_pepc_rates)

    model.add_cons_vars(trans_PEPC_cons)

    #PPDK constraint
    trans_PPDKs_M  = mm.get_rxn(model, 'trans_PPDKs_M')
    trans_PPDKs_BS = mm.get_rxn(model, 'trans_PPDKs_BS')
    wt_PPDKs_M = mm.get_rxn(model, 'PPDKs_M')
    wt_PPDKs_BS = mm.get_rxn(model, 'PPDKs_BS')
    
    trans_PPDKs_cons = model.problem.Constraint( 
        trans_PPDKs_BS.flux_expression + trans_PPDKs_M.flux_expression 
        +wt_PPDKs_BS.flux_expression + wt_PPDKs_M.flux_expression, 
                                             lb = 0, ub = trans_ppdks_rates)
    trans_PPDKs_cons.name = 'trans_ppdks_cons'
    model.add_cons_vars(trans_PPDKs_cons)


    #Malate Dehydrogenase Constraints
    trans_MDHm_M = mm.get_rxn(model, 'trans_MDHm_M')
    trans_MDHs_M = mm.get_rxn(model, 'trans_MDHs_M')
    trans_MDHs_BS = mm.get_rxn(model, 'trans_MDHs_BS')
    wt_MDHm_M =  mm.get_rxn(model, 'MDHm_M')
    wt_MDHs_M = mm.get_rxn(model, 'MDHs_M')
    wt_MDHs_BS = mm.get_rxn(model, 'MDHs_BS')
    
    trans_mdh_cons =  model.problem.Constraint(
       trans_MDHm_M.flux_expression + 
        wt_MDHm_M.flux_expression + 
        trans_MDHs_M.flux_expression + 
        trans_MDHs_BS.flux_expression +
        wt_MDHs_BS.flux_expression +
        wt_MDHs_M.flux_expression, 
        lb= 0, ub=trans_mdh_rates)

    trans_mdh_cons.name = "trans_mdh_cons"
    model.add_cons_vars(trans_mdh_cons)

    #Add NADP-ME constraints
    trans_MDHym_M = mm.get_rxn(model, 'trans_MDHym_M')
    wt_MDHys_M = mm.get_rxn(model, 'MDHys_M')
    wt_MDHys_BS = mm.get_rxn(model, 'MDHys_BS')
    
    trans_nadpme_cons = model.problem.Constraint(
        trans_MDHym_M.flux_expression + 
        wt_MDHys_M.flux_expression + 
        wt_MDHys_BS.flux_expression,
        lb= 0, ub=trans_nadp_me_rates)
    
    trans_nadpme_cons.name = "trans_nadpme"
    model.add_cons_vars(trans_nadpme_cons)

    #Add carbonic anhydrase constraints

    trans_hco3ec_M = mm.get_rxn(model, 'trans_hco3ec_M')
    wt_hco3ec_M = mm.get_rxn(model, 'HCO3Ec_M')
    wt_hco3em_M = mm.get_rxn(model, 'HCO3Em_M')
    wt_hco3es_M = mm.get_rxn(model, 'HCO3Es_M')
    wt_hco3ec_BS = mm.get_rxn(model, 'HCO3Ec_BS')
    wt_hco3em_BS = mm.get_rxn(model, 'HCO3Em_BS')
    wt_hco3es_BS = mm.get_rxn(model, 'HCO3Es_BS')
    
    trans_ca_cons = model.problem.Constraint(trans_hco3ec_M.flux_expression + 
                                             wt_hco3es_M.flux_expression + 
                                             wt_hco3ec_M.flux_expression + 
                                             wt_hco3em_M.flux_expression + 
                                             wt_hco3es_BS.flux_expression + 
                                             wt_hco3ec_BS.flux_expression + 
                                             wt_hco3em_BS.flux_expression,
                                      lb = -trans_CA_rates, ub = trans_CA_rates)
    trans_ca_cons.name = 'Trans_CA_cons'
    model.add_cons_vars(trans_ca_cons)
    model.repair()
#     print('Successfully added Transgenic-specific constraints')

SETUPS:
Setup 1: singular solution for WT model
Setup 2: Singular solution for Trans model

In [4]:
#WT Model
test_model = wt_model

with test_model as m:
    ppfd = 25
    test_model.medium = define_model_medium(m, co2=29, o2=inf, ppfd=ppfd, h=inf, nh4=inf, no3=inf)
    turn_off_cofac_cycles(m)
    add_tissue_constraints(m)
    add_enzyme_constraints(m)
    add_ngam_cons(m, ppfd)
    
    #Turn on all Demand reactions for protons and see how it goes

    mm.get_rxn(m,'Straw_Biomass_M').objective_coefficient = 0
    mm.get_rxn(m,'Straw_Biomass_BS').objective_coefficient = 0
    mm.get_rxn(m, 'DM_Phloem_BS').objective_coefficient = 1

    test_model = wt_model.copy()
    
    
test_model.metabolites.get_by_id('co2_s0').summary()
    
#This codeblock is used to generate solutions and check summaries for individual reactions/metabolites

Read LP format model from file /tmp/tmp5342fl58.lp
Reading time = 0.08 seconds
: 3970 rows, 9886 columns, 43002 nonzeros


Percent,Flux,Reaction,Definition
0.01%,6.023E-06,ARGDHTs_M,arogenate_s0 + h_s0 --> co2_s0 + h2o_s0 + phe-L_s0
0.01%,7.55E-06,ARGDHs_M,arogenate_s0 + nadp_s0 --> co2_s0 + nadph_s0 + tyr-L_s0
98.00%,0.09227,CO2ts_M,co2_c0 <=> co2_s0
1.98%,0.001868,ORPDCs_M,h_s0 + orot5p_s0 --> co2_s0 + ump_s0
Percent,Flux,Reaction,Definition
1.98%,-0.001868,CBMKs_M,atp_s0 + co2_s0 + nh4_s0 <=> adp_s0 + cbp_s0 + 2.0 h_s0
98.02%,-0.09229,RBPCs_M,co2_s0 + h2o_s0 + rb15bp_s0 --> 2.0 3pg_s0 + 2.0 h_s0


In [ ]:
# Trans model test
# add_trans_reactions(trans_model)

test_model = trans_model

with test_model as m:
    ppfd = 100
    test_model.medium = define_model_medium(m, co2=22.2, o2=inf, ppfd=ppfd, h=inf, nh4=inf, no3=inf)
    turn_off_cofac_cycles(m)
    add_tissue_constraints(m)
    add_enzyme_constraints(m)
    add_trans_constraints(m)
    add_ngam_cons(m, ppfd)
    
    #Turn on all Demand reactions for protons and see how it goes

    mm.get_rxn(m,'Straw_Biomass_M').objective_coefficient = 0
    mm.get_rxn(m,'Straw_Biomass_BS').objective_coefficient = 0
    mm.get_rxn(m, 'DM_Phloem_BS').objective_coefficient = 1
    
    test_model = trans_model.copy()
    

#Use this line to generate a solution specific to a single compound in the moudel    
test_model.metabolites.get_by_id('atp_c0').summary()
# test_model.reactions.trans_MDHym_M
#This codeblock is used to generate solutions and check summaries for individual reactions/metabolites

In [ ]:
##### TEST: What if we added a constraint to TPI (Triose Phosphate Utilization) based on TPU?
test_model = wt_model

with test_model as m:
    ppfd = 1500
    test_model.medium = define_model_medium(m, co2=29, o2=inf, ppfd=ppfd, h=inf, nh4=inf, no3=inf)
    turn_off_cofac_cycles(m)
    add_tissue_constraints(m)
    add_enzyme_constraints(m)
    add_ngam_cons(m, ppfd)
    
    # Add TPU constraints

    DHAP_transloc = test_model.reactions.get_by_id('DHAP(pi)tsr_M')
    _3PG_transloc = test_model.reactions.get_by_id('3PG(pi)tsr_M')

    TPU_cons = test_model.problem.Constraint(DHAP_transloc.flux_expression + _3PG_transloc.flux_expression,                                                  
                                                  
                                             lb = -10, ub = 10)
    TPU_cons.name = 'TPU_constraint'
    test_model.add_cons_vars(TPU_cons)
    # test_model.reactions.get_by_id('3PG(pi)tsr_M').bounds = (-inf,inf)

    #Turn on all Demand reactions for protons and see how it goes

    mm.get_rxn(m,'Straw_Biomass_M').objective_coefficient = 0
    mm.get_rxn(m,'Straw_Biomass_BS').objective_coefficient = 0
    mm.get_rxn(m, 'DM_Phloem_BS').objective_coefficient = 1

    test_model = wt_model.copy()
    
    # cobra.flux_analysis.parsimonious.add_pfba(test_model)
    
    
test_model.metabolites.get_by_id('co2_s0').summary()


#Results: No need as TPU flux is already 

In [ ]:
test_model.reactions.get_by_id('DHAP(pi)tsr_M')

In [ ]:
test_model.reactions.get_by_id('3PG(pi)tsr_M')

NOTES (16/03/2023)
- Trans model has decreased flux towards Glycine Dehydrogenase while having increased twice the flux towards Citrate synthesis.
- Due to this, there is less CO2 being exported by the Mitochondria for re-fixing as compared to the Wild type CO2 
- This may explain the increased detected flux towards citrate in Trans models.
- Without constraining Glycine dehydrogenase, a decrease in flux is detected in this reaction

- This has minor effect in terms of mitochondrial CO2 export in high light (i.e. above 1000 PPFD)
- Effect on Citrate flux is more pronounced at Low  light levels (~100 PPFD) where ICDH reaction is x8 times more in Trans model than in WT model. 
 


QUESTION: What would be the effect if Glycine Dehydrogenase flux is then further reduced to 0? (i.e. knockout experiment) -- I've written this script already, I need to analyze the solutions (Apr. 12, 2023)


Notes (Mar 23, 2023)
LOW LIGHT
- Small amounts of flux toward malate is detected and is exported to the BS cell and is subsequently transported mainly to the stroma and peroxisome via the Malate-Glutamate antiporter (Stromal) and the Malate-Oxaloacetate antiporter (Peroxisomal), respectively.

In the Stroma, it is then exported out via the AKG-Malate antiporter back into the cytosol while AKG is used for glutamate synthesis rather than towards CO2 fixation. This implies that a malate shuttle is functioning in a c3 system with a locally-expressed NADP-ME C4 enzyme. However, this cycle is truncated and the malate being shuttled is preferentially used for Glutamate synthesis instead.

Interestingly, CO2 availability is augmented in the BS stroma with the release of CO2 from anaplerotic reactions such as Pyruvate Dehydrogenase, Diaminoopimelate decarboxylase and others also related with Amino acid biosynthesis. Such a phenomenon is not observed in the stromal compartment of the M cell and instead is mostly supplied by atmospheric CO2 absorption and Mitochondrial CO2 production.

ASPARTATE EXPORT FROM THE M CELL

Another alternative pathway to C4 photosynthesis found in C3-C4 intermediate species is the Aspartate shuttle. In this case, Aspartate export from the M cell is observed in the model parametrized with the Trans reactions at light levels around 500 ppfd above. Most of the Aspartate (~50 percent) is exported to the Chloroplast via the H-Asp symporter (Stromal) while the rest is either exported as a photoassimilate (via the Phloem demand reaction) and  piped towards reactions specific for AA biosynthesis either in the cytosol or stroma. In the stroma, in particular, the majority of the transported Aspartate is relegated to the Phosphorylated pathway of Serine Biosynthesis (PPSB), w/c produces threonine and cysteine.


No such occurence is observed in the WT model even at varying light levels.



PYRUVATE EXPORT TO THE BS CELL

In both cases, Pyr export into the BS Cell was observed in both WT and Trans-parametrized models, with the latter having a higher export rate than the Trans model. However, it was observed that most of the exported Pyruvate is used as a nitrogen acceptor for the generation of Alanine.


Based on these, it is then observed that the fluxes of ketoacids and carbon intermediates is relegated mostly towards Amino acid biosynthesis, which is consistent with the findings of Hua et al 2021 where Bundle Sheath cells in rice were found to be preconditioned towards both solute transport (for phloem loading), water transport and Amino acid Biosynthesis. This inference was made by checking the enrichment patterns of the transcriptomes found in each of the excised tissue fractions corresponding to the BS and M cells.


NOTES: April 10, 2023

MALATE EXPORT TO THE BS  CELL:

It was observed under high light that flux of Malate through the PD is observed at a minimal flux level (~0.31 umol/m2s). However, this does not occur in all solutions generated by the Trans model and may only occur in certain flux distributions. This may be further detected using Flux Sampling to see the probability distribution of the flux values.

Interestingly, the fate of Malate in the BS cell is observed to be shunted towards the Stroma, where it is converted into AKG and subsequently used for Glutamate biosynthesis (Glutamate synthase  in Stroma). IN the Peroxisome, roughly a third of the transported malate is converted  to Oxaloacetate, which is then utilized in a sort of futile cycling that shuttles Malate and Oxaloacetate back to the cytosol without any end products. However, it is apparent that it may be a mechanism to maintain NAD/NADH balance within the Peroxisome, which in turn is utilized in the partial C2 cycle to oxidize H-pyruvate to R-glycerate within the compartment.

This cycle is found to be more apparent in the Trans-parametrized model than in the WT model where almost 40 percent of the malate in the BS cell is shuttled towards the peroxisome.

Note that only a small percentage of cytosolic Malate in the BS cell is derived from the M Cell and much of it is derived from the interconversion of Alanine and Glutamate -- which in turn are transported 



From Charton et al (2019):
"Smaller solutes with molecular masses below 300–400 Da, like the organic acids malate, oxaloacetate, and 2‐oxoglutarate, are shuttled via non‐selective channels across the peroxisomal membrane."



Note that in the Line 29 used to parametrize  the Transgenic model, the NADP-ME decarboxylating enzyme is ectopically expressed in the M mitochondria, which prevents Malate from being used in the Calvin Cycle in the BS cell. This is consistent with the results by Ermakova et al (2021) wherein the expression of NADP-ME was the least consistent of all the enzymes that were inserted to rice and that no decarboxylation was observed in vivo. However, observing the flux distributions within the M mitochondrial cell reveal the utilization of both transgenic MDHs (NADP and NAD). 

Interestingly, there is a shift in the utilization of either MDHs based on light conditions, wherein the NAD-utilizing MDH is predominant at lower light levels while the NADP-utilizing MDH (which produces CO2) is inactive at low light while active at high light levels.

For recommendation: Deactivating flux towards the OAA-Mal shuttle may allow more Malate to be shuttled to the Stroma instead of being cycled towards the peroxisome. I need to check whether this is active in the BS cell using the transcriptomic data from Hua et al (2021). Interestingly enough it doesn't have any annotated GPRs but I may try to find it by finding a reference for it. 



QUESTION: Is the Glycine Shuttle predicted to be active in the WT and Trans models?

Answer: Based on the flux distributions and metabolite summaries shown above, it is predicted to be active in both WT and Trans models. It is expected to be exported as a photoassimilate under low to medium light conditions. Furthermore, sensitivity analysis 







NOTES ON ANAPLEROTIC EXPORT OF CO2 and Re-fixation

NOTES ON Triose Phosphate Utilization Constraint
Triose Phosphate Utilization is one of the three canonical biological limitations of photosynthesis wherein the assimilation of carbon is limited by the ability to regenerate phosphate. Triose Phosphate efflux from the Chloroplast is represented by the Triose Phosphate Translocator (3PG(pi)tsr) which encodes the transport reaction translocating 3-Phosphoglycerate from the Cytosol to the Stromal compartment

Based on the flux values generated for the light scans, the total Triose Phosphate efflux from the Chloroplast did not exceed the experimentally derived values for both Trans and WT models. Hence, no constraint was implemented 





Conclusions:
- Increased shuttling of Ketoacids, particularly malate, was observed in the Trans-parametrized model compared to the WT model.
- Utilization of the inserted Enzymes is observed, but the ectopic expression of NADP-ME prevents in the MT the crucial step in the release of CO2 into the BS cell and being used in the 




#Side effects of Rebalancinng the reactions:
- Consummption of O2 does not cause biomass generation anymore, unlike before. Furthermore, it also prevents biomass formation if H+ is limited beyond a certain point
- 